In [1]:
import script.pyPLUTO as pp
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np

In [2]:
mpl.rcParams["figure.dpi"] = 150
FOLDER = "data"
N = 19

In [3]:
rho = []
vx = []
vy = []
prs = []

for i in range(N):
    D = pp.pload(i, w_dir=FOLDER+"/")
    rho.append(D.rho)
    vx.append(D.vx1)
    vy.append(D.vx2)
    prs.append(D.prs)

FileNotFoundError: [Errno 2] No such file or directory: 'data/data.0000.dbl'

In [ ]:
MAX = 0
MIN = 10

plotvar = prs

for i in range(10):
    m = np.min(plotvar[i])
    M = np.max(plotvar[i])
    if (m<MIN):
        MIN=m
    if M>MAX:
        MAX=M

In [ ]:
for i in range(N):
    fig, ax = plt.subplots(1,1)
    
    #im=ax.imshow(np.log(plotvar[i]), vmin=np.log(MIN), vmax=np.log(MAX), cmap="plasma")
    im=ax.imshow(plotvar[i], vmin=MIN, vmax=MAX, cmap="plasma")

    fig.colorbar(im, ax=ax)

    plt.title("time: %.4f s"%(i*0.0005))
    
    fig.savefig("images/prs-linear%04i.png"%i)
    
    plt.close(fig=fig)
    
    if i%10 == 0:
        print(i)